***Assignment 2***
by Magnus A. Nielsen (s204072) & Philip J. F. Helsted (s204147)
GitHub:
**Contribution statement**
all work has been done cooperatively

In [37]:
# imports
import numpy as np
import pandas as pd
import networkx as nx
import encoding

In [38]:
# Loading graph from week4
css_graph = encoding.read_json_file("data/graph.json")

# Setting random seed
np.random.seed(79)

**Exercise 1**
Mixing Patterns and Assortativity

In [39]:
# For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.
avg = []
for node in css_graph.nodes:
    frac = 0
    n_neighbours = 0
    for neighbour in nx.neighbors(css_graph, node):
        n_neighbours += 1
        try:
            if css_graph.nodes[node]['top_field'] == css_graph.nodes[neighbour]['top_field']:
                frac += 1
        except KeyError:
            pass
    avg.append(frac/n_neighbours)
avg = np.mean(avg)
print(avg)

0.21465140738302269


In [44]:
# Create a new graph, with the same nodes and edges, but where the association between nodes and field is shuffled.
# Compute the measure above for this randomized graph.


5652